In [ ]:
import sys
sys.path.append('..')
%env CUDA_VISIBLE_DEVICES=0

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import torch
import torch.nn as nn
import os
from collections import defaultdict

from torch.distributions import MultivariateNormal

from src.mrartemev_nflib.flows import NormalizingFlowModel, InvertiblePermutation, Invertible1x1Conv, ActNorm, NSF_AR
from src.mrartemev_nflib.flows import MAF, AffineHalfFlow
from src.mrartemev_nflib.nn import ARMLP, MLP

from torch.utils.data import Dataset, DataLoader, TensorDataset
from itertools import repeat

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

from src.nf import CalibratedModel, neg_log_likelihood
from src.nf.classifiers import train_catboost_clf
from scipy.special import logsumexp, expit


os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

In [ ]:
! ls dumps_20200602/GAS/SPLINE-AR_2_ind1

In [ ]:
def fix_act_norm(layer):
    if isinstance(layer, ActNorm):
        layer.data_dep_init_done = True

In [ ]:
def get_model(model_type, data, num_layers, dump_path):
    flows = []
    for _ in range(num_layers):
        if model_type == 'MAF':
            flows.append(MAF(dim=data.n_dims, base_network=ARMLP))
            flows.append(InvertiblePermutation(dim=data.n_dims))
        if model_type == 'SPLINE-AR':
            flows.append(ActNorm(dim=data.n_dims))
            flows.append(Invertible1x1Conv(dim=data.n_dims))
            flows.append(NSF_AR(dim=data.n_dims, K=8, B=3, hidden_features=32, depth=1, base_network=MLP))
        if model_type == 'GLOW':
            flows.append(ActNorm(dim=data.n_dims))
            flows.append(Invertible1x1Conv(dim=data.n_dims))
            flows.append(AffineHalfFlow(dim=data.n_dims, hidden_features=32, base_network=MLP))
            flows.append(InvertiblePermutation(dim=data.n_dims))
        if model_type == 'RealNVP':
            flows.append(AffineHalfFlow(dim=data.n_dims, base_network=MLP))
            flows.append(InvertiblePermutation(dim=data.n_dims))

    lr = 0.0005

    prior = MultivariateNormal(torch.zeros(data.n_dims).to(device), torch.eye(data.n_dims).to(device))
    model = NormalizingFlowModel(prior, flows).to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    checkpoint = torch.load(dump_path)
    model.load_state_dict(checkpoint['model.state_dict()'])
    model.apply(fix_act_norm)
    
    return model

In [ ]:
def to_device(model, device):
    model.to(device)
    model.prior = MultivariateNormal(torch.zeros(data.n_dims).to(device),
                                     torch.eye(data.n_dims).to(device))

In [ ]:
def repeater(data_loader):
    for loader in repeat(data_loader):
        for data in loader:
            yield data

In [ ]:
def batched_sample(model, n, batch_size=14000):
    generated = []
    for _ in range(n // batch_size):
        generated_batch = model.sample(batch_size)
        generated.append(generated_batch.cpu().detach())
    if n % batch_size != 0:
        generated_batch = model.sample(n % batch_size)
        generated.append(generated_batch.cpu().detach())
    generated = torch.cat(generated, dim=0)
    assert n == len(generated)
    return generated

In [ ]:
def logloss_with_logits(y_pred_logits, y_true):
    return -np.mean(
        y_true * np.logaddexp(0, -y_pred_logits) + \
        (1 - y_true) * np.logaddexp(0, y_pred_logits)
    )

In [ ]:
def train_cb(model, X_train_tensor, X_test_tensor, clips, iters):
    clf_ds_train = np.row_stack([
        np.column_stack([X_train_tensor.cpu().detach().numpy(), np.ones(len(X_train_tensor)).reshape(-1, 1)]),
        np.column_stack([model.sample_n(len(X_train_tensor)).cpu().detach().numpy(), np.zeros(len(X_train_tensor)).reshape(-1, 1)])
    ]).astype(np.float32)

    clf_ds_test = np.row_stack([
        np.column_stack([X_test_tensor.cpu().detach().numpy(), np.ones(len(X_test_tensor)).reshape(-1, 1)]),
        np.column_stack([model.sample_n(len(X_test_tensor)).cpu().detach().numpy(), np.zeros(len(X_test_tensor)).reshape(-1, 1)])
    ]).astype(np.float32)

    samples = model.sample_n(n).cpu().detach().cpu().numpy()
    

    metrics = []
    for n_iters in iters:
        clf = CatBoostClassifier(n_iters, verbose=0).fit(
            clf_ds_train[:, :-1], clf_ds_train[:, -1],
        )
        for clip in clips:
            calibrated_model = CalibratedModel(
                lambda x: np.clip(clf.predict(x, prediction_type='RawFormulaVal'), -100, clip),
                model,
                logit=True
            )
            clf_preds = np.clip(clf.predict(samples, prediction_type='RawFormulaVal'), -100, np.log(clip))
            calibration_constant = logsumexp(clf_preds) - np.log(len(clf_preds))
            logits = clf_preds - calibration_constant
            ll = -neg_log_likelihood(calibrated_model, X_test_tensor.cpu().detach()) - calibration_constant
            auc_roc = roc_auc_score(
                clf_ds_test[:, -1],
                np.clip(clf.predict(clf_ds_test[:, :-1], prediction_type='RawFormulaVal'), -100, np.log(clip)),
            )
            log_loss = logloss_with_logits(
                np.clip(clf.predict(clf_ds_test[:, :-1], prediction_type='RawFormulaVal'), -100, np.log(clip)),
                clf_ds_test[:, -1]
            )
            metrics.append({
                'clip': clip,
                'iters': n_iters,
                'll': ll,
                'auc_roc': auc_roc,
                'logloss': log_loss,
                'overhead': np.max(logits),
                'calibration_constant': calibration_constant
            })

    return clf_ds_train, clf_ds_test, metrics

In [ ]:
from utils import data_utils

data_mapping = {'BSDS300': data_utils.BSDS300,
                'GAS': data_utils.GAS,
                'MINIBOONE': data_utils.MINIBOONE,
                'POWER': data_utils.POWER,
                'HEPMASS': data_utils.HEPMASS}

In [ ]:
def get_best_model(model_type, data, num_layers, dumps_path):
    best_ll, best_model, best_dump = -10000000, None, None
    X_test_tensor = torch.from_numpy(data.tst.x[:100000]).to(device)
    
    for dump_path in [dumps_path + '/final_model.checkpoint'] + [
        os.path.join(dumps_path, 'checkpoints', path) for path in os.listdir(dumps_path + '/checkpoints')
    ]:
        try:
            model = get_model(model_type, data, num_layers, dump_path)
        except FileNotFoundError:
            print(f'Not found {dump_path}')
            continue
        ll = -neg_log_likelihood(model, X_test_tensor)
        if ll > best_ll:
            best_ll = ll
            best_model = model
            best_dump = dump_path
    return best_model, best_ll, best_dump

In [ ]:
arr = []
for data_name in ('MINIBOONE', 'BSDS300', 'GAS', 'HEPMASS', 'POWER'):
    data = data_mapping[data_name]()
    dim = data.n_dims
    n = min(100000, data.trn.x.shape[0])
    X_train_tensor = torch.from_numpy(data.trn.x[:n]).to(device)
    X_test_tensor = torch.from_numpy(data.tst.x[:n]).to(device)

    for model_type in ('GLOW', 'MAF', 'RealNVP', 'SPLINE-AR'):
        num_layers = 2 if model_type == 'SPLINE-AR' else 5
        model_name = f"{model_type}_{num_layers}"

        dumps_path = f'dumps_20200602/{data_name}/{model_type}_{num_layers}_ind1'
        model, ll, dump_path = get_best_model(model_type, data, num_layers, dumps_path)
        
        model.eval()
        model.sample_n = lambda n: batched_sample(model, n)
        to_device(model, 'cpu')

        clf_ds_train, clf_ds_test, metrics = train_cb(model, X_train_tensor, X_test_tensor, [10000, 2, 1.5], [20, 100, 500, 1000, 5000])
        
        arr.append({
            'data_name': data_name,
            'model_type': model_type,
            'll': ll,
            'metrics': metrics,
            'dump_path': dump_path,
        })
        print(data_name, model_type, ll, [x['ll'] for x in metrics])
#         print(arr[-1])

In [ ]:
[x['dump_path'] for x in arr]

In [ ]:
# metrics = defaultdict(dict)
# for x in arr:
#     metrics[x['data_name']][x['model_type']] = x['ll']
#     for y in x['metrics']:
#         if clip['clip'] == 100:
#             metrics[x['data_name']][x['model_type'] + ' C ' + str(y['iters'])] = y['ll']
#         else:
#             metrics[x['data_name']][x['model_type'] + ' C ' + str(round(y['clip'], 1)) + ' ' + str(y['iters'])] = y['ll']
# pd.DataFrame(metrics)

In [70]:
metrics = defaultdict(dict)
for x in arr:
    model_name = (x['model_type'], 'normalizing flow', '', '', )
    metrics[(x['data_name'], 'll', )][model_name] = x['ll']
    metrics[(x['data_name'], 'log overhead', )][model_name] = 0
    for y in x['metrics']:
        if y['clip'] == 10000:
            model_name = (x['model_type'], 'calibrated', y['iters'], 'inf', )
            metrics[(x['data_name'], 'll', )][model_name] = y['ll']
            metrics[(x['data_name'], 'log overhead', )][model_name] = y['overhead']
            metrics[(x['data_name'], 'AUC-ROC', )][model_name] = y['auc_roc']
            metrics[(x['data_name'], 'Logloss', )][model_name] = -y['logloss']
#             metrics[(x['data_name'], 'calib const', )][(x['model_type'], 'calibrated', )] = clip['calibration_constant']
        else:
            model_name = (x['model_type'], 'calibrated', y['iters'], round(y['clip'], 2), )
            metrics[(x['data_name'], 'll', )][model_name] = y['ll']
            metrics[(x['data_name'], 'log overhead', )][model_name] = y['overhead']
            metrics[(x['data_name'], 'AUC-ROC', )][model_name] = y['auc_roc']
            metrics[(x['data_name'], 'Logloss', )][model_name] = -y['logloss']
#             metrics[(x['data_name'], 'calib const', )][(x['model_type'], ' calibrated clip ' + str(round(clip['clip'], 2)), )] = clip['calibration_constant']
metrics = pd.DataFrame(metrics)
pd.set_option('display.max_rows', metrics.shape[0] + 1)
metrics

MINIBOONE                         \
                                            ll log overhead   AUC-ROC   
GLOW      normalizing flow          -14.054896     0.000000       NaN   
          calibrated       20   inf   0.447676    10.106144  1.000000   
                                2    -5.183384     7.564656  0.999863   
                                1.5  -5.543525     7.416833  0.999863   
                           100  inf   3.379196     9.648936  1.000000   
                                2    -4.758100     7.989940  0.999726   
                                1.5  -5.120303     7.840055  0.999726   
                           500  inf   1.776171     9.459289  1.000000   
                                2    -4.590256     8.157784  0.999863   
                                1.5  -4.987380     7.972978  0.999863   
                           1000 inf   2.595455     9.077106  1.000000   
                                2    -4.433438     8.314603  0.999863   
                                1.5  -4.824058     8.136300  0.999863   
                           5000 inf   1.363892     9.872564  1.000000   
                                2    -4.564393     8.183647  0.999863   
                                1.5  -4.935206     8.025152  0.999863   
MAF       normalizing flow          -12.341055     0.000000       NaN   
          calibrated       20   inf -11.338033     4.223443  0.806649   
                                2   -11.462662     0.974175  0.801030   
                                1.5 -11.507094     0.741330  0.793160   
                           100  inf -11.031978     5.301167  0.825287   
                                2   -11.277167     1.046320  0.815484   
                                1.5 -11.310846     0.844059  0.803250   
                           500  inf -11.034269     4.343867  0.843661   
                                2   -11.253903     1.062465  0.835812   
                                1.5 -11.291468     0.847141  0.823711   
                           1000 inf -11.039384     4.178808  0.846656   
                                2   -11.243158     1.067636  0.839017   
                                1.5 -11.283034     0.850345  0.827837   
                           5000 inf -10.996782     4.350558  0.851253   
                                2   -11.213632     1.078947  0.843355   
                                1.5 -11.254682     0.863763  0.832070   
RealNVP   normalizing flow          -13.369072     0.000000       NaN   
          calibrated       20   inf   6.879388     9.912973  1.000000   
                                2    -2.985626     9.076594  1.000000   
                                1.5  -3.370101     8.904437  1.000000   
                           100  inf  10.791322     9.752113  1.000000   
                                2    -1.920831     9.752113  1.000000   
                                1.5  -2.420831     9.752113  1.000000   
                           500  inf  10.665385     8.967242  1.000000   
                                2    -0.844860     8.967242  1.000000   
                                1.5  -1.344860     8.967242  1.000000   
                           1000 inf  10.203266     9.686474  1.000000   
                                2    -1.148991     9.686474  1.000000   
                                1.5  -1.648991     9.686474  1.000000   
                           5000 inf  10.323008     9.962048  1.000000   
                                2    -1.446498     9.962048  1.000000   
                                1.5  -1.946498     9.962048  1.000000   
SPLINE-AR normalizing flow          -18.083902     0.000000       NaN   
          calibrated       20   inf -16.938779     4.291992  0.828822   
                                2   -17.089095     0.976703  0.819601   
                                1.5 -17.111925     0.767233  0.807445   
                           100  inf -16.640777     5.306528  0.847329   
                      